In [1]:
import pandas as pd
import numpy as np
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

In [2]:
df_jan = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
df_feb = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

In [3]:
#Q1 Read the data for January. How many columns are there?
len(df_jan.columns)

19

In [4]:
#Q2 What's the standard deviation of the trips duration in January?
df_jan['duration'] = df_jan.tpep_dropoff_datetime - df_jan.tpep_pickup_datetime
df_jan.duration = df_jan.duration.apply(lambda td: td.total_seconds() / 60)
np.std(df_jan['duration'])

42.5943442974141

In [5]:
#Q3 What fraction of the records left after you dropped the outliers?
df_jan_out = df_jan[(df_jan.duration >= 1) & (df_jan.duration <= 60)]
len(df_jan_out)/len(df_jan)

0.9812202822125979

In [6]:
categorical = ['PULocationID', 'DOLocationID']

df_jan_out[categorical] = df_jan_out[categorical].astype(str)
train_dicts = df_jan_out[categorical].to_dict(orient='records')

/tmp/ipykernel_16139/3397381160.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_jan_out[categorical] = df_jan_out[categorical].astype(str)


In [7]:
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

target = 'duration'
y_train = df_jan_out[target].values

In [8]:
#Q4 What's the dimensionality of this matrix (number of columns)?
X_train.shape

(3009173, 515)

In [9]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

In [10]:
#Q5 Training a model. RMSE on train
np.sqrt(mean_squared_error(y_train, y_pred))

7.6492610279057605

In [11]:
df_feb['duration'] = df_feb.tpep_dropoff_datetime - df_feb.tpep_pickup_datetime
df_feb.duration = df_feb.duration.apply(lambda td: td.total_seconds() / 60)
df_feb_out = df_feb[(df_feb.duration >= 1) & (df_feb.duration <= 60)]
df_feb_out[categorical] = df_feb_out[categorical].astype(str)
val_dicts = df_feb_out[categorical].to_dict(orient='records')

/tmp/ipykernel_16139/2315544888.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_feb_out[categorical] = df_feb_out[categorical].astype(str)


In [12]:
#Evaluating the model. RMSE on validation
X_val = dv.transform(val_dicts)
y_val = df_feb_out[target].values
y_pred = lr.predict(X_val)
np.sqrt(mean_squared_error(y_val, y_pred))

7.81183265470218